In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/CS6910/Assignmnet3_Q8


/content/drive/MyDrive/CS6910/Assignmnet3_Q8


In [4]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 35.7 MB/s 
     |████████████████████████████████| 880 kB 39.8 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e71cf75fe690be41225a2aa669e695d83ad587d51cb7375e211f087e717d8556
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import numpy as np
import csv
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import random
from tqdm import tqdm, trange

**Handeling the CSV files**

"dataset 2" (kaagle) is choosen for this task. Link: https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres .

The two 'lyrics-data.csv' and 'artists-data.csv' were stored at : content/drive/Mydrive/CS6910/Assignmnet3_Q8 .

Choose songs from available languages and genres as per the artist selection. 

We have choosen - english songs ('en') - genre ('Pop' , 'Rock', 'Romântico') - popularity rating (> '4') .

In [19]:
lyrics_df = pd.read_csv('./lyrics-data.csv')
artists_df = pd.read_csv('./artists-data.csv')

lyrics_df = lyrics_df[lyrics_df['language']=='en'] #choose english song only

artists_df = artists_df[(  (artists_df['Genres'].isin(['Romântico'])) |  (artists_df['Genres'].isin(['Pop'])) | (artists_df['Genres'].isin(['Rock'])) ) & (artists_df['Popularity']>4) ]

#merging lyrics_df with ALink to artists_df with Link
lyrics_artists_df = lyrics_df.merge(artists_df[['Artist', 'Genres', 'Link']], left_on='ALink', right_on='Link', how='inner')

#drop unnecessary columns 
lyrics_artists_df = lyrics_artists_df.drop(columns=['ALink','SLink','Link','language'])

#print(len(lyrics_artists_df))


**Preparing Train and evaluation SET for Pop, Rock and Romântico**

In [20]:
pop_song_df = lyrics_artists_df[(lyrics_artists_df['Genres'].isin(['Pop']))]
rock_song_df = lyrics_artists_df[(lyrics_artists_df['Genres'].isin(['Rock']))]
rom_song_df = lyrics_artists_df[(lyrics_artists_df['Genres'].isin(['Romântico']))]

train_eval_ratio = 0.9
pop_train, pop_eval = train_test_split(pop_song_df, train_size = train_eval_ratio, random_state = 1)
rock_train, rock_eval = train_test_split(rock_song_df, train_size = train_eval_ratio, random_state = 1)
rom_train, rom_eval = train_test_split(rom_song_df, train_size = train_eval_ratio, random_state = 1)

#resetting index in both sets
pop_train = pop_train.reset_index()
pop_train = pop_train.drop(columns=['index', 'Genres'])
pop_eval = pop_eval.reset_index()
pop_eval = pop_eval.drop(columns=['index', 'Genres'])

rock_train = rock_train.reset_index()
rock_train = rock_train.drop(columns=['index', 'Genres'])
rock_eval = rock_eval.reset_index()
rock_eval = rock_eval.drop(columns=['index', 'Genres'])

rom_train = rom_train.reset_index()
rom_train = rom_train.drop(columns=['index', 'Genres'])
rom_eval = rom_eval.reset_index()
rom_eval = rom_eval.drop(columns=['index', 'Genres'])

Working with Romântico genre for now.


In [21]:
print(f"Romântico - Training set")
display(rom_train)

print(f"Romântico - Evaluation set")
display(rom_eval)

Romântico - Training set


,SName,Lyric,Artist
0,The Scene,"The Scene\n(Graham Russell, Mark Towner Willia...",Air Supply
1,Don't Throw Our Love Away,DON'T THROW OUR LOVE AWAY\n\nOh I'm just wonde...,Air Supply
2,Those Dancing Days Are Gone,"Come, let me sing into your ear;\nThose dancin...",Carla Bruni
3,The Little Drummer Boy,Come they told me pa rum pum pum pum\nA new bo...,Air Supply
4,Sweet Dreams,This is the time when you need a friend\nYou j...,Air Supply
...,...,...,...
196,Until,The shore would never reach as far\nAs once be...,Air Supply
197,Me Like You,"Me like you, Me like you, Me like you, Me like...",Air Supply
198,Dancing With The Mountain,"(Graham Russell, Michael Sherwood, David Young...",Air Supply
199,Mumbo Jumbo,It’s a thousand miles from anywhere you know\n...,Air Supply


Romântico - Evaluation set


,SName,Lyric,Artist
0,Faith,"(Rick Hahn, George Thatcher)\n\nEasy come and ...",Air Supply
1,Always,(Graham Russell/Guy Allison/Michael Sherwood)\...,Air Supply
2,Cherry Blossom Girl,I don't want to be shy\nCan't stand it anymore...,Air Supply
3,I COME ALIVE (date with an Angel),I COME ALIVE (date with an Angel)\n\nEveryday ...,Air Supply
4,Come What May,"(Cynthia Weil, Tom Snow)\n\nWhen she looks at ...",Air Supply
5,I Know You Better Than You Think,(Graham Russell)\n\nYou didn't have to be so m...,Air Supply
6,Can I Be Your Lover?,"Rest you head on the Earth\nFor to me, you are...",Air Supply
7,Now And Forever,(Graham Russell)\n\nWhen love is new\nAnd the ...,Air Supply
8,Daybreak,"Daybreak, I can't hide the truth\nIt seems I'm...",Air Supply
9,Absolute Beginners,I've nothing much to offer\nThere's nothing mu...,Carla Bruni


Worked with Romântico dataframe

In [22]:
# taking Romântico into consideration
GENRE = "Romântico"


# in {?_com} replace ? by (rom/ rock/ pop)
test_set = rom_eval
train_set = rom_train

# the idea is to split the Lyric into "Lyric" and "End_lyric" and while 
#        evaluation predicting the end lyric as per the trained model

#taking n characters from the end as End_Lyric
last_N_characters = -20

test_set['End_lyric'] = test_set['Lyric'].str.split().str[last_N_characters:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:last_N_characters].apply(' '.join)

print(f"Genre : {GENRE}")
display(test_set)

Genre : Romântico


,SName,Lyric,Artist,End_lyric
0,Faith,"(Rick Hahn, George Thatcher) Easy come and eas...",Air Supply,You got to have faith And you lie awake starin...
1,Always,(Graham Russell/Guy Allison/Michael Sherwood) ...,Air Supply,run dry Here in a moment of time passing by Yo...
2,Cherry Blossom Girl,I don't want to be shy Can't stand it anymore ...,Air Supply,That means no time to waste Whenever there's a...
3,I COME ALIVE (date with an Angel),I COME ALIVE (date with an Angel) Everyday I p...,Air Supply,warm You give me strength to carry on I know I...
4,Come What May,"(Cynthia Weil, Tom Snow) When she looks at me ...",Air Supply,fingers When they just can't be found I turn a...
5,I Know You Better Than You Think,(Graham Russell) You didn't have to be so mad ...,Air Supply,"than you think and always will, always will I ..."
6,Can I Be Your Lover?,"Rest you head on the Earth For to me, you are ...",Air Supply,Will you be my love? Can I be your lover? Will...
7,Now And Forever,(Graham Russell) When love is new And the worl...,Air Supply,here with you I'm here with you I'm holding on...
8,Daybreak,"Daybreak, I can't hide the truth It seems I'm ...",Air Supply,won't you come to stay I waited far too long a...
9,Absolute Beginners,I've nothing much to offer There's nothing muc...,Carla Bruni,the films There's no reason To feel all the ha...


Adding "I love Deep learning" into the test dataframe 

In [23]:
df = pd.DataFrame()
dict = {'SName': 'Test_Custom', 'Lyric': 'I love deep learning', 'Artist' : 'Rohit & Aditya', 'End_lyric' : 'Custom'}
df = df.append(dict, ignore_index = True)

frames = [test_set, df]  
result = pd.concat(frames)

test_set = result.reset_index()
test_set = test_set.drop(columns=['index'])

display(test_set)


,SName,Lyric,Artist,End_lyric
0,Faith,"(Rick Hahn, George Thatcher) Easy come and eas...",Air Supply,You got to have faith And you lie awake starin...
1,Always,(Graham Russell/Guy Allison/Michael Sherwood) ...,Air Supply,run dry Here in a moment of time passing by Yo...
2,Cherry Blossom Girl,I don't want to be shy Can't stand it anymore ...,Air Supply,That means no time to waste Whenever there's a...
3,I COME ALIVE (date with an Angel),I COME ALIVE (date with an Angel) Everyday I p...,Air Supply,warm You give me strength to carry on I know I...
4,Come What May,"(Cynthia Weil, Tom Snow) When she looks at me ...",Air Supply,fingers When they just can't be found I turn a...
5,I Know You Better Than You Think,(Graham Russell) You didn't have to be so mad ...,Air Supply,"than you think and always will, always will I ..."
6,Can I Be Your Lover?,"Rest you head on the Earth For to me, you are ...",Air Supply,Will you be my love? Can I be your lover? Will...
7,Now And Forever,(Graham Russell) When love is new And the worl...,Air Supply,here with you I'm here with you I'm holding on...
8,Daybreak,"Daybreak, I can't hide the truth It seems I'm ...",Air Supply,won't you come to stay I waited far too long a...
9,Absolute Beginners,I've nothing much to offer There's nothing muc...,Carla Bruni,the films There's no reason To feel all the ha...


In [24]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', from_pt=True)
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Tokenizing the data
class Tokenized_Lyric(Dataset):  
    def __init__(self, code):
        self.song_lyrics = []
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        
        #maximum limit to the lyric length in chracters
        character_length=1024
        
        def __len__(self):
          count_lyrics = len(self.song_lyrics)
          return self.count_lyrics

        for lyric in train_set['Lyric']:
          self.song_lyrics.append(torch.tensor(self.tokenizer.encode(f"<|{code}|>{lyric[:character_length]}<|endoftext|>")))               
        
Tokenized_dataset = Tokenized_Lyric(train_set['Lyric'])

**Pre-training GPT2 model**

In [25]:
def Pre_train(dataset, model, tokenizer, batch_size, no_of_epochs, learning_rate, max_seq_len=400, warmup_steps=200):
    
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    #using adam as optimizer with modular learning rate
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    
    #warmup_steps - to lower the learning rate in order to reduce the impact of model deviation 
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1)
    
    train_dataloader = DataLoader(Tokenized_dataset.song_lyrics, batch_size=1, shuffle=True)
    
    in_tensor = None
    loss=0
    acc_batch_count = 0


    for epoch in range(no_of_epochs):

        print(f"Pre-training epoch {epoch}")
        print(loss)
        
        for idx, entry in tqdm(enumerate(train_dataloader)): 
          #Accumulated batch size    
            if in_tensor is None:
                in_tensor = entry
                carry_on = True
                remainder = None
            if entry.size()[1] + in_tensor.size()[1] > max_seq_len:
                carry_on = False
                remainder = entry
            else:
                in_tensor = torch.cat([entry, in_tensor[:, 1:]], dim=1)
                carry_on = True
                remainder = None

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            in_tensor = in_tensor.to(device)
            outputs = model(in_tensor, labels=in_tensor)
            loss = outputs[0]
            loss.backward()

            if (acc_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            acc_batch_count += 1
            in_tensor = None
        
    return model

model = Pre_train(Tokenized_dataset, model, tokenizer, batch_size =16, no_of_epochs=5, learning_rate=2e-5 )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Pre-training epoch 0
0


201it [01:04,  3.10it/s]


Pre-training epoch 1
tensor(3.0112, device='cuda:0', grad_fn=<NllLossBackward0>)


201it [01:04,  3.11it/s]


...

201it [01:04,  3.12it/s]


Pre-training epoch 3
tensor(3.2091, device='cuda:0', grad_fn=<NllLossBackward0>)


201it [01:04,  3.11it/s]


Pre-training epoch 4
tensor(2.5680, device='cuda:0', grad_fn=<NllLossBackward0>)


201it [01:04,  3.11it/s]


**Generation/Prediction of Lyrics**

In [26]:
#Function to generate multiple sentences. Test data should be a dataframe
def lyric_prediction(test_set):
  predicted_lyrics = []
  for i in range(len(test_set)):
    
    prompt =  test_set['Lyric'][i]
    entry_count=1
    
    #maximum_no_of words
    entry_length=30

    #model keeps most relevant word probabilities
    top_probabilty=0.9

    #high temperature forces the model to make more original predictions
    temp=1.0

    model.to('cpu').eval()
    predicted_count = 0
    predicted_list = []

    with torch.no_grad():

        for entry_index in trange(entry_count):

            entry_finished = False
            predicted_ = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(predicted_, labels=predicted_)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temp if temp > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probabilties = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probabilties > top_probabilty
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = -float("Inf")#Filter value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                predicted_ = torch.cat((predicted_, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:
                  predicted_count = predicted_count + 1
                  out_list = list(predicted_.squeeze().numpy())
                  out_text = tokenizer.decode(out_list)
                  predicted_list.append(out_text)
                  break
            
            if not entry_finished:
              out_list = list(predicted_.squeeze().numpy())
              out_text = f"{tokenizer.decode(out_list)}<|endoftext|>" 
              predicted_list.append(out_text)

    predicted_lyrics.append(predicted_list)
  return predicted_lyrics

lyrics_generated = lyric_prediction(test_set)


100%|██████████| 1/1 [00:05<00:00,  5.31s/it]


**Final predictions**

In [30]:
#storing predicted lyrics into new columnredicted_lyrics=[]
Predicted_lyrics=[]

for i in range(len(lyrics_generated)):
  a = test_set['Lyric'][i].split()[last_N_characters:] #Get the matching string we want
  b = ' '.join(a)
  c = ' '.join(lyrics_generated[i])
  Predicted_lyrics.append(c.split(b)[-1])

test_set['Predicted_lyric'] = Predicted_lyrics




## Uncomment to get ensure that the predicted lyrics ends at '.'
#ensuring predicted lyrics ends at '.'
#final_Predicted_lyrics=[]

#for i in range(len(test_set)):
#  to_remove = test_set['Predicted_lyric'][i].split('.')[-1]
#  final_Predicted_lyrics.append(test_set['Predicted_lyric'][i].replace(to_remove,''))
#test_set['Predicted_lyric'] = final_Predicted_lyrics

In [31]:
test_set

,SName,Lyric,Artist,End_lyric,Predicted_lyric
0,Faith,"(Rick Hahn, George Thatcher) Easy come and eas...",Air Supply,You got to have faith And you lie awake starin...,(You know that you have to wait Until all tha...
1,Always,(Graham Russell/Guy Allison/Michael Sherwood) ...,Air Supply,run dry Here in a moment of time passing by Yo...,grow silent (Source: Stevie Wonder)\n\nSkyscr...
2,Cherry Blossom Girl,I don't want to be shy Can't stand it anymore ...,Air Supply,That means no time to waste Whenever there's a...,Every night Chancer song plays And this is lo...
3,I COME ALIVE (date with an Angel),I COME ALIVE (date with an Angel) Everyday I p...,Air Supply,warm You give me strength to carry on I know I...,smile for all you have Give me your gift ever...
4,Come What May,"(Cynthia Weil, Tom Snow) When she looks at me ...",Air Supply,fingers When they just can't be found I turn a...,fingers And they're full of regret (Chorus) C...
5,I Know You Better Than You Think,(Graham Russell) You didn't have to be so mad ...,Air Supply,"than you think and always will, always will I ...",now than I can You saved me And never know\n\...
6,Can I Be Your Lover?,"Rest you head on the Earth For to me, you are ...",Air Supply,Will you be my love? Can I be your lover? Will...,Will you take my hand? Can I be your lover? W...
7,Now And Forever,(Graham Russell) When love is new And the worl...,Air Supply,here with you I'm here with you I'm holding on...,"afraid of you My heart says, But 'tis a pain ..."
8,Daybreak,"Daybreak, I can't hide the truth It seems I'm ...",Air Supply,won't you come to stay I waited far too long a...,can't I hold my head high? I just want to wal...
9,Absolute Beginners,I've nothing much to offer There's nothing muc...,Carla Bruni,the films There's no reason To feel all the ha...,"the films\nPosted by Carl From Bainbridge, Ke..."


In [32]:
display(test_set.iloc[-1])

SName                                                    Test_Custom
Lyric                                           I love deep learning
Artist                                                Rohit & Aditya
End_lyric                                                     Custom
Predicted_lyric     and the power of abstraction."\n\nPapers and ...
Name: 23, dtype: object

In [35]:
#BLEU score  End lyrics VS Predicted End lyrics

import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]
for i in range(len(test_set)):
  end_lyrics = test_set['End_lyric'][i]
  predicted_end_lyrics = test_set['Predicted_lyric'][i]
  scores.append(sentence_bleu(end_lyrics, predicted_end_lyrics))

statistics.mean(scores)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.60899222640997